Instructions

The file contains the adjacency list representation of a simple undirected graph.
There are 200 vertices labeled 1 to 200. The first column in the file represents
the vertex label, and the particular row (other entries except the first column)
tells all the vertices that the vertex is adjacent to. So for example, the 6th row
looks like: "6 155 56 52 120 ......". This just means that the vertex with label 6
is adjacent to (i.e., shares an edge with) the vertices with labels 155,56,52,120,......,etc

Your task is to code up and run the randomized contraction algorithm for the min
cut problem and use it on the above graph to compute the min cut. (HINT: Note that
you'll have to figure out an implementation of edge contractions. Initially, you
might want to do this naively, creating a new graph from the old every time there's
an edge contraction. But you should also think about more efficient implementations.)
(WARNING: As per the video lectures, please make sure to run the algorithm many times
with different random seeds, and remember the smallest cut that you ever find.)
Write your numeric answer in the space provided. So e.g., if your answer is 5, just
type 5 in the space provided.

In [1]:
# Store in the same way as adjacency list

import csv
def read_graph(filename):
    # Read an undirected graph in CSV format. Each line is an edge
    tsv = csv.reader(open(filename), delimiter='\t')
    graph = {}
    
    for node in tsv:
        graph[node[0]] = node[1:-1]
            
    return graph

from random import choice
def random_edge(graph):
    node1 = choice(graph.keys())
    node2 = choice(graph[node1])
    return node1, node2

def merge(graph, node1, node2):
    # remove node2 from the graph and change connections of
    # all other nodes that are connected to node2 to node1
    
    graph[node1].remove(node2)
    graph[node2].remove(node1)
    
    graph[node1].extend(graph[node2])
    
    for edge2 in set(graph[node2]):
        new_edges = []
        for node in graph[edge2]:
            if node == node2:
                new_edges.append(node1)
            else:
                new_edges.append(node)
        graph[edge2] = new_edges

    graph.pop(node2)
    
    return graph

def delete_self_loop(graph, node):
    if node in graph[node]:
        graph[node] = [i for i in graph[node] if i != node]
    return graph

def min_cut(graph):
    while len(graph) > 2:
        node1, node2 = random_edge(graph)
        graph = merge(graph, node1, node2)
        graph = delete_self_loop(graph, node1)
#         print node1, node2, graph
    m = len(graph[graph.keys()[0]])
    print m, graph.keys()
    return m

import copy
def main(graph):
    Nnodes = len(graph)
    cuts = []
    
    for i in range(Nnodes):
        G = copy.deepcopy(graph)
        m = min_cut(G)
        cuts.append(m)

    return min(cuts)

In [2]:
# test
graph = {1: [2, 3], 2: [1, 3, 4], 3: [1, 2, 4], 4: [2, 3]}
nodes = [1, 2, 3, 4]
results = []
main(graph)

3 [2, 4]
3 [2, 4]
3 [2, 3]
2 [2, 4]


2

In [3]:
def read_from_text(text):
    lines = [i for i in text.split("\n") if i]
    graph = {}
    
    for line in lines:
        nodes = line.split()
        graph[nodes[0]] = nodes[1:]
    
    return graph

text = """
1 2 3 4 7

2 1 3 4

3 1 2 4

4 1 2 3 5

5 4 6 7 8

6 5 7 8

7 1 5 6 8

8 5 6 7
""" 
graph = read_from_text(text)
main(graph)

6 ['4', '7']
4 ['1', '3']
4 ['5', '4']
3 ['3', '8']
4 ['5', '8']
3 ['3', '5']
3 ['1', '2']
2 ['4', '8']


2

In [ ]:
graph = read_graph("./kargerMinCut.txt")

from timeit import default_timer as timer
start = timer()

for i in range(1):
    main(graph)

end = timer()
print "Timing: {0:.6f} s".format(end - start)

In [4]:
# A different data structure where we store unique edges

import csv
def read_graph2(filename):
    # Read an undirected graph in CSV format. Each line is an edge
    tsv = csv.reader(open(filename), delimiter='\t')
    edges = []
    vertices = set()
    
    for nodes in tsv:
        for node in nodes[1:-1]:
            vertices.add(node)
            b1 = (nodes[0], node) not in edges
            b2 = (node, nodes[0]) not in edges
            if b1 and b2:
                edges.append((nodes[0], node))

    return edges, vertices

from random import choice
def random_edge2(edges):
    node1, node2 = choice(edges)
    return node1, node2

def merge2(edges, nodes, node1, node2):
    edges.remove((node1, node2))
    nodes.remove(node2)
    
    nedges = len(edges)
    for i in range(nedges):
        u, v = edges[i]
        if u == node2:
            edges[i] = (node1, v)
        if v == node2:
            edges[i] = (u, node1)
    
#     # test failure
#     for u, v in edges:
#         if u == node2 or v == node2:
#             print "MERGE FAIL", u, v
    
    return edges, nodes

def delete_self_loop2(edges):
    return [i for i in edges if i[0] != i[1]]

def min_cut2(edges, nodes):
    while len(nodes) > 2:
        node1, node2 = random_edge2(edges)
        edges, nodes = merge2(edges, nodes, node1, node2)
        edges = delete_self_loop2(edges)
    mc = len(edges)
    print mc, edges[0]
    return mc

import copy
def main2(edges, nodes):
    n_nodes = len(nodes)
    m = None
    
    for i in range(n_nodes):
        E = copy.deepcopy(edges)
        N = copy.deepcopy(nodes)
        if m == None:
            m = min_cut2(E, N)
        else:
            n = min_cut2(E, N)
            if n < m:
                m = n
    
    return m

In [5]:
def read_from_text2(text):
    lines = (i for i in text.split("\n") if i)
    edges = []
    vertices = set()
    
    for line in lines:
        nodes = line.split()
        for node in nodes[1:]:
            vertices.add(node)
            b1 = (nodes[0], node) not in edges
            b2 = (node, nodes[0]) not in edges
            if b1 and b2:
                edges.append((nodes[0], node))
    
    return edges, vertices

text = """
1 2 3 4 7

2 1 3 4

3 1 2 4

4 1 2 3 5

5 4 6 7 8

6 5 7 8

7 1 5 6 8

8 5 6 7
"""
 
edges, nodes = read_from_text2(text)
main2(edges, nodes)

4 ('1', '5')
5 ('1', '2')
3 ('2', '6')
4 ('1', '6')
3 ('1', '6')
2 ('1', '6')
4 ('2', '6')
2 ('1', '5')


2

In [ ]:
edges, nodes = read_graph2("./kargerMinCut.txt")

from timeit import default_timer as timer
start = timer()

main2(edges, nodes)

end = timer()
print "Timing: {0:.6f}".format(end - start)